In [72]:
# Mount drive
import os
from google.colab import drive
%cd /content
drive.mount('/content/drive', force_remount=True)
%cd 'drive/My Drive/Colab Notebooks/NLP/project/code'
%ls -l
print(os.listdir())

/content
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/NLP/project/code
total 68
-rw------- 1 root root  3534 Apr 22 16:32 cleaning.py
-rw------- 1 root root 21800 Apr 25 09:06 pytorch_tuto.ipynb
-rw------- 1 root root 22808 Apr 22 16:46 Untitled.ipynb
-rw------- 1 root root  3545 Apr 22 16:32 utils_cleaning.py
-rw------- 1 root root 16626 Apr 25 16:38 Vincent_w2v.ipynb
['cleaning.py', 'utils_cleaning.py', 'Untitled.ipynb', '.ipynb_checkpoints', 'pytorch_tuto.ipynb', '.vector_cache', 'Vincent_w2v.ipynb']


In [73]:
import pandas as pd
import numpy as np
import random
import re
import time
from string import punctuation
import matplotlib.pyplot as plt

# import nltk
# from nltk.corpus import stopwords
# nltk.download('stopwords')
# from nltk.stem import SnowballStemmer
# nltk.download('punkt')
# from nltk.stem import WordNetLemmatizer 
# nltk.download('wordnet')

# from gensim.test.utils import datapath, get_tmpfile
# from gensim.models import KeyedVectors
# from gensim.scripts.glove2word2vec import glove2word2vec

import torch
import torch.nn as nn
from torchtext.data import Field, LabelField, TabularDataset, Iterator, BucketIterator
from torchtext import vocab
import torch.optim as optim

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
EMBEDDING_PATH = '../data/embeddings/glove.6B.100d.txt'

In [75]:
pd.read_csv("../data/train.csv").head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [0]:
# Preprocessing

tokenize = lambda x: x.split()
TEXT = Field(sequential=True, tokenize=tokenize, lower=True)
LABEL = LabelField(dtype=torch.float)

In [0]:
train_datafields = [("id", None),
                 ("qid1", None),
                 ("qid2", None),
                 ("question1", TEXT),
                 ("question2", TEXT),
                 ("is_duplicate", LABEL)]

test_datafields = [("id", None),
                   ("question1", TEXT),
                   ("question2", TEXT)]

# trn, vld = TabularDataset(
#         path="../data/train.csv",
#         format='csv',
#         skip_header=True, # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
#         fields=tv_datafields
#         ).split(
#         split_ratio=0.7
#         )

In [78]:
train_data = TabularDataset(
        path="../data/train.csv",
        format='csv',
        skip_header=True, # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
        fields=train_datafields
        )

# test_data = TabularDataset(
#         path="../data/test.csv",
#         format='csv',
#         skip_header=True, # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
#         fields=test_datafields
#         )

train_data, valid_data = train_data.split(
                  split_ratio=0.9,
                  random_state=random.seed(42)
                  )

print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
# print(f'Number of test examples: {len(test_data)}')

Number of training examples: 363861
Number of validation examples: 40429


In [0]:
vec = vocab.Vectors(EMBEDDING_PATH)

TEXT.build_vocab(train_data, vectors = vec)
LABEL.build_vocab(train_data)

In [81]:
print(TEXT.vocab.freqs.most_common(10))
print(train_data[0].question1, train_data[0].question2)
print(valid_data[0].__dict__['question1'], valid_data[0].__dict__['question2'])
print(len(TEXT.vocab))

[('the', 338525), ('what', 279949), ('is', 242034), ('how', 197011), ('i', 191790), ('a', 189350), ('to', 183554), ('in', 175138), ('of', 142965), ('do', 139198)]
['how', 'do', 'i', 'loose', 'weight', 'fast?'] ['how', 'can', 'i', 'loose', 'weight', 'in', 'a', 'week?']
['what', 'are', 'the', 'difference', 'between', 'share', 'and', 'equity?'] ['what', 'is', 'the', 'difference', 'between', 'a', 'stock,', 'a', 'share', 'and', 'an', 'equity?']
189775


In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device='cpu'
train_iter, valid_iter = BucketIterator.splits(
        (train_data, valid_data), # we pass in the datasets we want the iterator to draw data from
        batch_size=64,
        device=device, # if you want to use the GPU, specify the GPU number here
        sort_key=lambda x: len(x.question1)+len(x.question2), # the BucketIterator needs to be told what function it should use to group the data.
        sort_within_batch=False
        # repeat=False # we pass repeat=False because we want to wrap this Iterator layer.
)

# test_iter = Iterator(
#     test,
#     batch_size=64,
#     device=device,
#     train=False,
#     sort=False,
#     sort_within_batch=False
# )

In [0]:
# i=0
# for batch in train_iter:
#   print(batch)
#   i+=1
#   if i==5:
#     break

In [0]:
class siameseNet(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
    super().__init__()
    # self.embedding = nn.EmbeddingBag(vocab_size, embedding_dim, mode='mean')
    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.linear = nn.Linear(embedding_dim, hidden_dim)

  def forward_one(self, x):
    output = self.embedding(x).mean(axis=0)
    # output = nn.AdaptiveAvgPool1d(1)(output)
    output = self.linear(output)
    return output

  def forward(self, input1, input2):
    emb1 = self.embedding(input1).mean(axis=0)
    emb2 = self.embedding(input2).mean(axis=0)
    output1 = self.linear(emb1)
    output2 = self.linear(emb2)
    # output = torch.norm(output1-output2)
    output = nn.CosineSimilarity()(output1, output2)
    return output

In [0]:
emb_shape = TEXT.vocab.vectors.shape
INPUT_DIM = emb_shape[0]
EMBEDDING_DIM = emb_shape[1]
HIDDEN_DIM = 50
OUTPUT_DIM = 1

model = siameseNet(vocab_size=INPUT_DIM, embedding_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM, output_dim=OUTPUT_DIM)

pretrained_embeddings = TEXT.vocab.vectors
# model.embedding.weight.data.copy_(pretrained_embeddings)
model.embedding.from_pretrained(pretrained_embeddings, freeze=True)

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
# criterion = nn.BCELoss()

model = model.to(device)
criterion = criterion.to(device)

In [0]:
def compute_accuracy(predictions, labels, thresh=0.5):
    predicted_labels = predictions >= thresh
    accuracy = (predicted_labels == labels).sum()/len(predictions)
    return accuracy.item()

def train(model, iterator, optimizer, criterion, thresh=0.5):
    # Track the loss
    epoch_loss = 0
    epoch_TP_FN = 0
    epoch_FP_TN = 0
    
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        
        predictions = model(batch.question1, batch.question2)
        loss = criterion(predictions, batch.is_duplicate)
        
        loss.backward()
        optimizer.step()

        predicted_labels = predictions >= thresh
        epoch_TP_FN += (predicted_labels==batch.is_duplicate).sum().item()
        epoch_FP_TN += (predicted_labels!=batch.is_duplicate).sum().item()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator), epoch_TP_FN/(epoch_TP_FN+epoch_FP_TN)


def evaluate(model, iterator, criterion, thresh=0.5):
    epoch_loss = 0
    epoch_TP_FN = 0
    epoch_FP_TN = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.question1, batch.question2)
            loss = criterion(predictions, batch.is_duplicate)

            predicted_labels = predictions >= thresh
            epoch_TP_FN += (predicted_labels==batch.is_duplicate).sum().item()
            epoch_FP_TN += (predicted_labels!=batch.is_duplicate).sum().item()

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator), epoch_TP_FN/(epoch_TP_FN+epoch_FP_TN)

In [86]:
N_EPOCHS = 6

# Track time taken
start_time = time.time()

for epoch in range(N_EPOCHS):
    epoch_start_time = time.time()
    
    train_loss, train_acc = train(model, train_iter, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iter, criterion)
    
    print(f'| Epoch: {epoch+1:02} '
          f'| Train Loss: {train_loss:.3f} '
          f'| Train Accuracy: {train_acc:.3f} '
          f'| Val. Loss: {valid_loss:.3f} '
          f'| Val. Accuracy: {valid_acc:.3f} '
          f'| Time taken: {time.time() - epoch_start_time:.2f}s'
          f'| Time elapsed: {time.time() - start_time:.2f}s')

| Epoch: 01 | Train Loss: 0.719 | Train Accuracy: 0.683 | Val. Loss: 0.724 | Val. Accuracy: 0.687 | Time taken: 101.26s| Time elapsed: 101.26s
| Epoch: 02 | Train Loss: 0.681 | Train Accuracy: 0.744 | Val. Loss: 0.702 | Val. Accuracy: 0.718 | Time taken: 101.83s| Time elapsed: 203.09s
| Epoch: 03 | Train Loss: 0.649 | Train Accuracy: 0.782 | Val. Loss: 0.685 | Val. Accuracy: 0.729 | Time taken: 102.28s| Time elapsed: 305.37s


KeyboardInterrupt: ignored